<a href="https://colab.research.google.com/github/gaurikathakur/Metro-crowd-detection/blob/main/metro_crowd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [2]:
import os, zipfile, shutil
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from google.colab import files


uploaded = files.upload()
zip_path = next(iter(uploaded))
extract_path = "/content/full_dataset"
os.makedirs(extract_path, exist_ok=True)


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted:", os.listdir(extract_path))
dataset_folder = os.path.join(extract_path, os.listdir(extract_path)[0])
print(f"Dataset folder: {dataset_folder}")


base_dir = "/content/dataset"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

def split_data(class_name):
    class_path = os.path.join(dataset_folder, class_name)
    images = os.listdir(class_path)
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

for cls in ['heavy', 'normal', 'less']:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    split_data(cls)


img_height, img_width = 224, 224
batch_size = 8

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
).flow_from_directory(train_dir, target_size=(img_height, img_width), class_mode='categorical', batch_size=batch_size)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=(img_height, img_width), class_mode='categorical', batch_size=batch_size)


base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(train_gen, validation_data=val_gen, epochs=10)


model.save("metro_crowd_model.keras")

Saving metro_crowd_dataset.zip to metro_crowd_dataset.zip
Extracted: ['metro_crowd_dataset']
Dataset folder: /content/full_dataset/metro_crowd_dataset
Found 14 images belonging to 3 classes.
Found 5 images belonging to 3 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.4048 - loss: 1.4551 - val_accuracy: 0.6000 - val_loss: 0.9442
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 500ms/step - accuracy: 0.6587 - loss: 0.7695 - val_accuracy: 0.6000 - val_loss: 0.9910
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 482ms/step - accuracy: 0.6508 - loss: 0.7051 - val_accuracy: 0.6000 - val_loss: 1.0146
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 453ms/step - accuracy: 0.7460 - loss: 0.5717 - val_accuracy: 0.6000 - val_loss: 0.9972
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 459ms/step - accuracy: 0.9524 - loss: 0.4196 - val_accuracy: 0.8000 - val_loss: 0.9005
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 414ms/step - accuracy: 1.0000 - loss: 0.3550 - val_accuracy: 0.8000 - val_loss: 0.8330
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step - accuracy: 0.8968 - loss: 0.2904 - val_accuracy: 0.8000 - val_loss: 0.8059
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step - accuracy: 0.9524 - loss: 0.2996 - val_accuracy: 0.8000 - val_loss: 0.

In [3]:
!pip install -q gradio gTTS tensorflow pillow


import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
from gtts import gTTS


model = load_model("metro_crowd_model.keras")


class_labels = ['Heavy Crowd', 'Less Crowd', 'Normal Crowd']
sections = ['Ticket Counter Area', 'Security Checking Zone', 'Platform Area', 'Inside the Coach']


def preprocess_image(image):
    image = image.resize((224, 224)).convert('RGB')
    img_array = np.array(image) / 255.0
    return np.expand_dims(img_array, axis=0)


def predict_crowd(ticket, security, platform, inside):
    inputs = [ticket, security, platform, inside]
    section_map = dict(zip(sections, inputs))

    report = "## 🧾 Delhi Metro Crowd Report\n\n"
    audio_text = "Here is your crowd status report for Delhi Metro. "

    for section, file in section_map.items():
        if file is not None:
            processed = preprocess_image(file)
            prediction = model.predict(processed)[0]
            pred_index = int(np.argmax(prediction))
            pred_label = class_labels[pred_index]
            confidence = prediction[pred_index] * 100

            report += f"### 🚩 {section} — **{pred_label}** ({confidence:.2f}%)\n"
            audio_text += f"In the {section.lower()}, the situation is {pred_label.lower()}. "


            if section == "Ticket Counter Area" and pred_label == "Heavy Crowd":
                report += "💡 *Suggestion:* Deploy additional ticket vending machines.\n"
                audio_text += "You can add extra ticket booths. "
            elif section == "Security Checking Zone" and pred_label == "Heavy Crowd":
                report += "💡 *Suggestion:* Increase number of security checkpoints.\n"
                audio_text += "Security staff can be increased. "
            elif section == "Inside the Coach" and pred_label == "Heavy Crowd":
                report += "💡 *Suggestion:* Consider running more frequent trains or better ventilation.\n"
                audio_text += "Try adding more trains for comfort. "

            report += "---\n"
        else:
            report += f"⚠️ {section}: No image uploaded.\n---\n"
            audio_text += f"{section.lower()} image was not provided. "


    tts = gTTS(text=audio_text, lang='en')
    tts.save("report.mp3")

    return report, "report.mp3"


interface = gr.Interface(
    fn=predict_crowd,
    inputs=[
        gr.Image(label="🎫 Ticket Counter Area", type="pil"),
        gr.Image(label="🛂 Security Checking Zone", type="pil"),
        gr.Image(label="📍 Platform Area", type="pil"),
        gr.Image(label="🚇 Inside the Coach", type="pil"),
    ],
    outputs=[
        gr.Markdown(label="📋 Crowd Report"),
        gr.Audio(label="🔊 Voice Report")
    ],
    title="🧠 Smart Metro Crowd Detection System",
    description="Upload photos from different areas of a metro station and get intelligent crowd insights with suggestions and audio feedback. Built using TensorFlow + Gradio.",
    theme=gr.themes.Soft(),
    allow_flagging="never"
)


interface.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.9 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60ac722958e03d11d1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
